# DeepDream with Tensorflow

inspired by [Magnus Erik Hvass Pedersen's DeepDream Tutorial](https://github.com/Hvass-Labs/TensorFlow-Tutorials) <br> and the [Tensorflow DeepDream Tutorial](https://www.tensorflow.org/tutorials/generative/deepdream) 

## Concept

DeepDream is a computer vision program created by Alexander Mordvintsev. It uses a convolutional neural network to enhance patterns in images. The effect is similar to that of pareidolia, which refers to the tendency to incorrectly interpret objects. For example, seeing shapes in clouds, human faces where there are none, etc.

The program works by passing an image through a CNN, selecting a layer in the network and applying the gradient of the layer in order to amplify the patterns detected. Then, the resulting image is passed through the CNN again. This process is done iteratively, such that at each iteration the emergent pattern is further defined.

## Imported libraries

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import random
import math
import utility
from IPython.display import Image, display

# Image manipulation.
import PIL.Image
from scipy.ndimage.filters import gaussian_filter

This was developed using Python 3.5.2 (Anaconda) and TensorFlow version:

In [ ]:
tf.__version__

## CNN model  used

The Inception5h (V1) model was chosen as it reportedly yields better results and accepts images of any size as input. Any other CNN model can be used as well, with the corresponding modifications (the layers differ).

The model can be downloaded from tensorflow. A default directory to save the data-files is created if it does not exist.

In [ ]:
import inception5h
# inception.data_dir = 'inception/5h/'
inception5h.maybe_download()

The model is loaded.

In [ ]:
model = inception5h.Inception5h()

The Inception5h model has many layers that can be used for DeepDreaming. A list of the 12 most commonly used layers is available.

In [ ]:
len(model.layer_tensors)

Each layer is trained to see specific features. Lower-level layers see simple patterns such as lines and edges and higher-level layers see more complex things such as animals. Examples of layer visualizations are available at [this](https://github.com/ProGamerGov/Protobuf-Dreamer/wiki/Interesting-Layers-And-Channels) repository.

layer 1: colors\
layer 2: waves\
layer 3: lines\
layer 4: boxes\
layer 5: circles\
layer 6: eyes\
layer 7: faces, more eyes\
layer 8: dogs, fury animals, worms, reptiles\
layer 9: fish, frogs/reptilian eyes\
layer 10: monkeys, lizards, snakes, ducks, butterflies\
layer 11: birds, insects\
layer 12: hot air balloons, complex structures

In [ ]:
layer_tensors = model.layer_tensors
#layer_tensors

layer_names = model.layer_names
layer_names

## DeepDream Algorithm

### Tiled gradient

Large input images take a toll on memory usage and take a long time to be computed. An optimization technique is to split the image into tiles, then compute the gradient for each tile.

This results in hard edges between tiles. A way to deal with this is to choose tiles randomly at each iteration, such that no seams become visible.

A helper-function for determining an appropriate tile-size is provided.

In [ ]:
def get_tile_size(num_pixels, tile_size=400):
    """
    num_pixels is the number of pixels in a dimension of the image.
    tile_size is the desired tile-size.
    """

    # Get possible number of tiles (at least 1)
    num_tiles = max(1, int(round(num_pixels / tile_size)))
    
    # The actual tile-size.
    actual_tile_size = math.ceil(num_pixels / num_tiles)
    
    return actual_tile_size

This helper-function computes the gradient for an input image. The image is split into tiles and the gradient is calculated for each tile. The tiles are chosen randomly to avoid visible seams / lines in the final DeepDream image.

In [ ]:
def tiled_gradient(gradient, image, tile_size=400):
    # Initialize the image gradients to zero.
    grad = np.zeros_like(image)

    # Get x and y dimensions.
    x, y, _ = image.shape

    # Tile-size for the x-axis.
    x_tile_size = get_tile_size(num_pixels=x, tile_size=tile_size)

    # Tile-size for the y-axis.
    y_tile_size = get_tile_size(num_pixels=y, tile_size=tile_size)

    # Random start-position for the tiles on the x-axis between -3/4 and 
    # -1/4 of the tile-size. If the tiles are smaller than 1/4 of the image
    # the gradients are noisy.
    x_start = random.randint(-3*(x_tile_size // 4), -(x_tile_size // 4))

    while x_start < x:
        
        x_end = x_start + x_tile_size
        
        # Ensure the tile's start- and end-positions are valid.
        x_start_lim = max(x_start, 0)
        x_end_lim = min(x_start + x_tile_size, x)

        # Random start-position for the tiles on the y-axis.
        # The random value is between -3/4 and -1/4 of the tile-size.
        y_start = random.randint(-3*(y_tile_size // 4), -(y_tile_size // 4))

        while y_start < y:

            y_end = y_start + y_tile_size
            
            # Ensure the tile's start- and end-positions are valid.
            y_start_lim = max(y_start, 0)
            y_end_lim = min(y_start + y_tile_size, y)

            # Get the image-tile.
            img_tile = image[x_start_lim:x_end_lim,
                             y_start_lim:y_end_lim, :]

            # Create a feed-dict with the image-tile.
            feed_dict = model.create_feed_dict(image=img_tile)

            # Use TensorFlow to calculate the gradient-value.
            g = session.run(gradient, feed_dict=feed_dict)

            # Normalize the gradient for the tile. This is
            # necessary because the tiles may have very different
            # values. Normalizing gives a more coherent gradient.
            g /= (np.std(g) + 1e-8)

            # Store the tile's gradient at the appropriate location.
            grad[x_start_lim:x_end_lim,
                 y_start_lim:y_end_lim, :] = g
            
            # Advance the start-position for the y-axis.
            y_start = y_end

        # Advance the start-position for the x-axis.
        x_start = x_end

    return grad

### Gradient ascent.

This function calculates the gradient of the given layer of the model with regard to the input image. The gradient is then added to the input image so the mean value of the layer-tensor is increased. This process is repeated a number of times and amplifies whatever patterns the Inception model sees in the input image.

Helper-function to get the gradient of the chosen layer.

In [ ]:
def get_layer_gradient(layer_tensor):
    
    return model.get_gradient(layer_tensor)

In [ ]:
def gradient_ascent(layer_tensor, image,
                   num_iterations=10, step_size=3.0, tile_size=400,
                   show_gradient=False, color=False):
    """
    Parameters:
    layer_tensor: Tensor that will be maximized
    image: Input image used as the starting point.
    num_iterations: Number of optimization iterations to perform.
    step_size: Scale for each step of the gradient ascent.
    tile_size: Size of the tiles when calculating the gradient.
    show_gradient: Plot the gradient in each iteration.
    color: Let algorithm modify initial colors.
    """

    # Copy the image so we don't overwrite the original image.
    img = image.copy()
    
    print("Image before:")
    utility.plot_image(img)

    print("Processing image: ", end="")
    
    gradient = model.get_gradient(layer_tensor)
    
    for i in range(num_iterations):
        # Compute the gradient for the current image.
        # The gradient is used to enhance features detected by the chosen layer.
        grad = tiled_gradient(gradient=gradient, image=img, tile_size=tile_size)
        
        # Apply Gaussian filters to blur the image for better results.
        # Values may be changed; other filters can be applied as well.
        
        # sigma - blur amount in increasing order
        sigma = (i * 4.0) / num_iterations + 0.5
        
        # keep original colours by blurring the colour-channel
        if color:
            grad_smooth1 = gaussian_filter(grad, sigma=(sigma, sigma, 0.0))
            grad_smooth2 = gaussian_filter(grad, sigma=(sigma*2, sigma*2, 0.0))
            grad_smooth3 = gaussian_filter(grad, sigma=(sigma*0.5, sigma*0.5, 0.0))
        # or don't blur the colour channel and allow colours to be changed by the gradient
        else:
            grad_smooth1 = gaussian_filter(grad, sigma=sigma)
            grad_smooth2 = gaussian_filter(grad, sigma=sigma*2)
            grad_smooth3 = gaussian_filter(grad, sigma=sigma*0.5)
        
        # combine the filtered gradients to obtain a nice looking image
        grad = (grad_smooth1 + grad_smooth2 + grad_smooth3)

        # Scale the step-size according to the gradient-values.
        step_size_scaled = step_size / (np.std(grad) + 1e-8)

        # Update the image by following the gradient.
        img += grad * step_size_scaled

        if show_gradient:
            # Print statistics for the gradient.
            msg = "Gradient min: {0:>9.6f}, max: {1:>9.6f}, stepsize: {2:>9.2f}"
            print(msg.format(grad.min(), grad.max(), step_size_scaled))

            # Plot the gradient.
            utility.plot_gradient(grad)
        else:
            # Otherwise show a little progress-indicator.
            print(". ", end="")

    print()
    print("Image after:")
    utility.plot_image(img)
    
    return img

### Taking it up an octave.

Using `gradient_ascent()` on large inputs results in small patterns, all of the same granularity, low resolution and noisiness. 

This function downsamples the image several times and applies `gradient_ascent()` to each of these scales. This technique addresses all of the above-mentioned issues and speeds up the computation. The first iteration will produce a big pattern, while the following iterations will add additional details.

In [ ]:
def recursive_optimize(layer_tensor, image,
                       num_repeats=4, rescale_factor=0.7, blend=0.2,
                       num_iterations=10, step_size=3.0,
                       tile_size=400, color=False):
    """
    Parameters:
    gradient: Gradient of the chosen layer
    image: Input image used as the starting point.
    rescale_factor: Downscaling factor for the image.
    num_repeats: Number of times to downscale the image.
    blend: Factor for blending the original and processed images.

    Parameters passed to gradient_ascent():
    num_iterations: Number of optimization iterations to perform.
    step_size: Scale for each step of the gradient ascent.
    tile_size: Size of the tiles when calculating the gradient.
    color: Let algorithm modify initial colors.
    """

    # Downsample
    if num_repeats>0:
        # Blur the input image to prevent artifacts when downscaling.
        # sigma - blur amount (the colour-channel is not blurred 
        # as it would make the image gray.
        sigma = 0.5
        img_blur = gaussian_filter(image, sigma=(sigma, sigma, 0.0))

        # Downscale the image.
        img_downscaled = utility.resize_image(image=img_blur,
                                      factor=rescale_factor)
            
        # Recursive call to this function.
        # Subtract one from num_repeats and use the downscaled image.
        img_result = recursive_optimize(layer_tensor=layer_tensor,
                                        image=img_downscaled,
                                        num_repeats=num_repeats-1,
                                        rescale_factor=rescale_factor,
                                        blend=blend,
                                        num_iterations=num_iterations,
                                        step_size=step_size,
                                        tile_size=tile_size, color=color)
        
        # Upscale the resulting image back to its original size.
        img_upscaled = utility.resize_image(image=img_result, size=image.shape)

        # Blend the original and processed images.
        image = blend * image + (1.0 - blend) * img_upscaled

    print("Recursive level:", num_repeats)

    # Process the image using the DeepDream algorithm.
    img_result = gradient_ascent(layer_tensor=layer_tensor,
                                image=image,
                                num_iterations=num_iterations,
                                step_size=step_size,
                                tile_size=tile_size, color=color)
    
    return img_result

## TensorFlow Session

An interactive TensorFlow session is used to execute and to continue adding gradient functions to the computational graph.

In [ ]:
session = tf.InteractiveSession(graph=model.graph)

## Flower Bloom

The algorithm is applied to an image of a flower bloom.

In [ ]:
image = utility.load_image(filename='images/borrego.jpg')
utility.plot_image(image)

Choose a layer to maximize certain features. The 3rd layer was chosen here (index 2). It has 192 channels whose values can be maximized.

In [ ]:
layer_tensor = model.layer_tensors[2]
layer_tensor

Run the gradient ascent algorithm and show the gradient for each iteration. Note the visible artifacts in the seams between the tiles.

In [ ]:
img_result = gradient_ascent(layer_tensor, image,
                   num_iterations=15, step_size=6.0, tile_size=800,
                   show_gradient=False)

Save the resulting image.

In [ ]:
utility.save_image(img_result, filename='results/gradient_ascent_l3_num-it15_step-size6_borrego.jpg')

Run the `recursive_optimize()` function. This will create an image with larger patterns. The reason is that the patterns were initially created on a heavily downsampled image and then refined on the higher resolution images.

In [ ]:
img_result = recursive_optimize(layer_tensor=layer_tensor, image=image,
                 num_iterations=10, step_size=3.0, rescale_factor=0.6,
                 num_repeats=5, blend=0.2)

In [ ]:
utility.save_image(img_result, filename='results/octaves_l3_num-it10_step-size3_rescale0-6_num-repeats5_blend0-2_borrego.jpg')

Choose another layer to maximize, for example layer 7 (index 6). This layer recognizes more complex shapes in the input image.

In [ ]:
layer_tensor = model.layer_tensors[6]
img_result = recursive_optimize(layer_tensor=layer_tensor, image=image,
                 num_iterations=10, step_size=4.0, rescale_factor=0.5,
                 num_repeats=4, blend=0.2)

In [ ]:
utility.save_image(img_result, filename='results/octaves_l7_num-it10_step-size4_rescale0-5_num-repeats4_blend0-2_borrego.jpg')

One can also maximize only a subset of a layer's feature-channels. For example, layer with index 7 and only its first 5 feature-channels.

In [ ]:
layer_tensor = model.layer_tensors[7]
layer_tensor

In [ ]:
layer_tensor = model.layer_tensors[7][:,:,:,0:5]
img_result = recursive_optimize(layer_tensor=layer_tensor, image=image,
                 num_iterations=10, step_size=3.0, rescale_factor=0.7,
                 num_repeats=4, blend=0.2)

In [ ]:
utility.save_image(img_result, filename='results/octaves_l8_c0-5_num-it10_step-size3_rescale0-7_num-repeats4_blend0-2_borrego.jpg')

Maximize the final layer.

In [ ]:
layer_tensor = model.layer_tensors[11]
img_result = recursive_optimize(layer_tensor=layer_tensor, image=image,
                 num_iterations=10, step_size=3.0, rescale_factor=0.5,
                 num_repeats=4, blend=0.2)

In [ ]:
utility.save_image(img_result, filename='results/octaves_l12_num-it10_step-size3_rescale0-5_num-repeats4_blend0-2_borrego.jpg')

The algorithm is optimized in such a way as to mostly keep the original image's colours. This is done by applying gaussian filters to the colour channels of the tiled images gradient. However, the algorithm can 'dream' colours as well and deviate from the original by not applying the filter to the channels as well. This can be easily done by setting the colour parameter to true as seen below.

In [ ]:
layer_tensor = model.layer_tensors[11]
layer_tensor

In [ ]:
layer_tensor = model.layer_tensors[11][:,:,:,120:125]
img_result = recursive_optimize(layer_tensor=layer_tensor, image=image,
                 num_iterations=10, step_size=6.0, rescale_factor=0.7,
                 num_repeats=5, blend=0.2, color=True)

In [ ]:
utility.save_image(img_result, filename='results/octaves_l12_c120-125_num-it10_step-size6_rescale0-7_num-repeats5_blend0-2_color_borrego.jpg')

## Close TensorFlow Session

Close the session to release its resources.

In [ ]:
session.close()